# <center> **Pengantar OpenMC** </center> 

**Dhiya Salma Salsabila/24923304**

### **Pendahuluan**
**OpenMC** mensimulasikan partikel berupa **neutron dan foton** yang **bergerak secara stokastik (acak)** melalui sebuah **model yang di definisikan secara arbitrer (bebas)**.

<p>

Dalam reaktor nuklir, **neutron menjadi sangat penting karena dapat menyebabkan reaksi fisi**. Dengan mengetahui perilaku neutron maka dapat **menentukan seberapa sering reaksi fisi terjadi**. Hal ini juga **sebanding dengan energi yang dihasilkan**. 

#### **Deterministrik**
Dalam metode ini diselesaikan dengan persamaan yang dapat **menggambarkan perilaku rata-rata partikel** atau biasa di sebut dengan **persamaan transport neutron**
<p>

Persamaan ini merupakan **persamaan berdimensi tujuh (tiga dimensi untuk ruang, tiga untuk kecepatan, dan satu untuk waktu)** yang sangat sulit diselesaikan secara langsung. Untuk semua kasus, kecuali masalah yang paling sederhana, diperlukan beberapa bentuk diskretisasi. 
<p>

**Diskretisasi dilakukan dengan membagi seluruh ruang menjadi bagian-bagian kecil yang homogen**, kemudian menyelesaikan persamaan pada bagian-bagian kecil tersebut. Setelah melakukan diskretisasi dan berbagai pendekatan lainnya, kita dapat menghasilkan bentuk persamaan yang sesuai untuk diselesaikan dengan komputer. **Di antaranya adalah metode ordinat diskret, metode karakteristik, difusi beda hingga, dan metode nodal (pemecahan diferensial parsial)**.
<p>

**Kelebihan:**

- Kecepatan solusi: Tergantung pada metode yang digunakan, solusi dapat ditentukan dengan sangat cepat.
- Solusi global: Solusi yang dihasilkan adalah solusi global, artinya kita mengetahui perilaku rata-rata di seluruh domain.
- Kepastian solusi: Setelah masalah mencapai konvergensi, solusinya diketahui dengan pasti.
<p>

**Kekurangan:**

- Kerumitan model kompleks: Jika modelnya rumit, diperlukan pembuatan mesh yang canggih dan kompleks.
- Persiapan penampang multi-grup: Diperlukan penampang multi-grup yang memerlukan pengetahuan solusi sebelumnya (a priori).

#### **Monte Carlo**
Dalam metode ini menentukan perilaku rata-rata berbagai kuantitas fisik dalam suatu sistem
<p>

**Kelebihan:**

- Tanpa pembuatan mesh: Tidak diperlukan pembuatan mesh untuk membangun geometri. Dengan menggunakan constructive solid geometry (CSG), dimungkinkan untuk membuat model kompleks dengan permukaan melengkung.
- Fleksibilitas dalam penampang energi: Metode Monte Carlo dapat digunakan baik dengan penampang energi kontinu maupun multi-grup.
- Kemudahan simulasi paralel: Menjalankan simulasi secara paralel secara konsep sangat sederhana.
<p>

**Kekurangan:**

- Biaya komputasi tinggi: Karena bergantung pada pengambilan sampel acak berulang, metode ini memerlukan komputasi yang sangat mahal.
- Tidak otomatis memberikan solusi global: Simulasi tidak secara otomatis memberikan solusi global di seluruh domain – Anda harus secara spesifik meminta kuantitas yang diinginkan.
- Ketidakpastian stokastik: Bahkan setelah masalah mencapai konvergensi, diperlukan simulasi banyak partikel untuk mengurangi ketidakpastian stokastik.

#### **Satuan dalam OpenMC**
- Panjang : centimeter
- Energi : electronvolt
- Waktu : second

## <Center> **Konfigurasi Data Nuklir** </Center>
OpenMC menggunakan format HDF5, namun jika kita memiliki data dalam format ACE yang telah di olah NJOY seperti yang biasa di pakai dalam MCNP atau Serpent maka data tersebut harus di konversi terlebih dahulu ke HDF5.

##### **Mengolah format ACE**

In [ ]:
import openmc
# Contoh
u235 = openmc.data.IncidentNeutron.from_ace('92235.710nc')
u235.export_to_hdf5('U235.h5')

Jika terdapat beberapa file dengan format ACE untuk nuklida yang sama tetapi suhu yang berbeda maka dapat digunakan 

In [ ]:
# Interaksi neutron, untuk simulasi interaksi neutron dengan bahan pada berbagai suhu
u235 = openmc.data.IncidentNeutron.from_ace('92235.710nc')
for suffix in [711, 712, 713, 714, 715, 716]:
    u235.add_temperature_from_ace('92235.{}nc'.format(suffix))
u235.export_to_hdf5('U235.h5')

# Hamburan termal, untuk simulasi interaksi neutron dengan bahan pada suhu rendah, di mana efek hamburan termal signifikan
light_water = openmc.data.ThermalScattering.from_ace('lwtr.20t')
for suffix in range(21, 28):
    light_water.add_temperature_from_ace('lwtr.{}t'.format(suffix))
light_water.export_to_hdf5('lwtr.h5')

Setelah semua file ACE sudah di ubah ke dalam HDF5, maka file harus di ekspor ke dalam XML

In [ ]:
library = openmc.data.DataLibrary()
library.register_file('U235.h5')
library.register_file('lwtr.h5')
...
library.export_to_xml()

#### **Mengolah dari ENDF**

In [ ]:
# Contoh penambahan Zr-92, temperatur default 293.6 K
zr92 = openmc.data.IncidentNeutron.from_njoy('n-040_Zr_092.endf')

# Contoh penambahan dengan beberapa temperatur
zr92 = openmc.data.IncidentNeutron.from_njoy(
    'n-040_Zr_092.endf', temperatures=[300., 600., 1000.])

#### **Photon Cross Section**

In [ ]:
# Dari ACE
u = openmc.data.IncidentPhoton.from_ace('92000.12p')
u.export_to_hdf5('U.h5')

# Dari ENDF
u = openmc.data.IncidentPhoton.from_endf('photoat-092_U_000.endf',
                                         'atom-092_U_000.endf')

## <Center> **Running OpenMC** </Center>
untuk menjalanlan OpenMC, hal utama yang harus dilakukan ialah membuat file XML yang menggambarkan model yang akan disimulasikan. Ada 3 file yang diperlukan :
- **materials.xml**, file ini berisi **material apa yang ada dalam model** dan dari apa material tersebut terdiri. Selain itu, file ini menunjukkan di mana OpenMC harus mencari perpustakaan penampang lintang.
- **geometry.xml**, file ini menjelaskan bagaimana material yang **didefinisikan dalam materials.xml menempati wilayah ruang**. Volume fisik didefinisikan menggunakan geometri padat konstruktif, dijelaskan secara rinci dalam Mendefinisikan Geometri.
- **settings.xml**, file ini menunjukkan **mode apa yang harus dijalankan OpenMC**, berapa banyak partikel yang akan disimulasikan, definisi sumber, dan berbagai opsi lainnya.
- **tallies.xml**, file ini menjelaskan **kuantitas fisik apa yang harus dihitung selama simulasi (fluks, laju reaksi, arus, dll.)**.
- **plots.xml**, file ini memberikan **spesifikasi untuk menghasilkan plot irisan atau voxel dari geometri**.

In [ ]:
# Create materials
materials = openmc.Materials()
...
materials.export_to_xml()

# Create geometry
geometry = openmc.Geometry()
...
geometry.export_to_xml()

# Assign simulation settings
settings = openmc.Settings()
...
settings.export_to_xml()

### **Komposisi Material**
